# Assigment 6

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

**GET YOUR GOOGLE API AND TOKEN. YOU WILL NEED THEM TO DO THIS TASK.**


1. Import Data from [this url](https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf). This dataset is in PDF format. You have to convert to PandasDataFrame and keep only the BBVA offices in LIMA.
2. Use GoogleMaps API and geocode all the BBVA offices. For those offices that Google API gets no information, use internet and get the latitude and longitude handly and add them to dataset.
3. Use Google API to find the driving time (best guess) from all the group members' address and all the LIMA BBVA offices.
4. Finally, you have to give a report which offices are the most closest and furthest to every group member's address.

## Solution

### 7.1 Installation and import of libraries

This code read the data from de excel and import the data in a Data Frame. 

In [1]:
#Install google maps function - just at the beggining
#pip install -U googlemaps

In [2]:
import pandas as pd
import numpy as np
import os
import urllib.request, json, csv
import googlemaps
from datetime import datetime
import io
import requests
from functools import reduce
import time
from tqdm import tqdm

In [3]:
# Read data from shared excel
oficinas_bbva_lima = pd.read_excel( r"../../_data/bbva_list.xlsx" )
oficinas_bbva_lima

,Direccion,DEPARTAMENTO,PROVINCIA,DISTRITO
0,CENTRO AEREO COMERCIAL LOCALES 110 A Y 111 A,LIMA,LIMA,CALLAO
1,AV. CTRMTE. MORA S/N BASE NAVAL,LIMA,LIMA,CALLAO
2,"AV. ELMER FAUCETT Y ALEJANDRO BERTELLO, CC CAN...",LIMA,LIMA,CALLAO
3,AV. SAENZ PEN A 323,LIMA,LIMA,CALLAO
4,CALLE OMEGA 149 PARQUE INDUSTRIAL DEL CALLAO,LIMA,LIMA,CALLAO
5,AV. CONTRALMIRANTE RAYGADA N°lll,LIMA,LIMA,CALLAO
6,"AV. ELMERT FAUCETT N°2121 LOCALES N° 2-101,2-1...",LIMA,LIMA,CALLAO
7,AV. ELMER FAUCETT 6000,LIMA,LIMA,CALLAO
8,CENTRO COMERCIAL MINKA PABELL6N 2,LIMA,LIMA,CALLAO
9,"AV. OSCAR R. BENAVIDES 3866, URB. EL AGUILA, L...",LIMA,LIMA,CALLAO


### 7.2 GoogleMaps API and geocode all the BBVA offices

This code geocodes all BBVA branches with the GoogleMaps API. Also in cases without data, the code looks for latitude and longitude and adds them to the dataset.

In [4]:
# Data you want to add in the new column
oficinas = [
    "AEROPUERTO", "BASE NAVAL", "C.C. CANTA CALLAO", "CALLAO", "COLONIAL",
    "EL PUERTO", "LIMA CARGO CITY", "OVALO 200 MILLAS", "MINKA",
    "MALL AVENTURA PLAZA CALLAO", "BELLAVISTA", "GUARDIA CHALACA", "REPSOL",
    "VENTANILLA", "BACKUS", "SALAMANCA", "NICOLAS AYLLON", "PURUCHUCO",
    "C.C REAL PLAZA SANTA CLARA", "VITARTE", "HUAYCAN", "BARRANCO",
    "PLAZA BUTTERS", "VENEZUELA I", "VENEZUELA III", "BREÑA", "CARABAYLLO",
    "CEDROS DE VILLA", "CHORRILLOS", "CLUB REGATAS LIMA", "PLAZA LIMA SUR",
    "VILLA MARINA", "COMAS", "MALL PLAZA COMAS", "SAN FELIPE COMAS",
    "V. A. BELAUNDE", "EL AGUSTINO", "MEGA PLAZA", "MULTIPLAZA",
    "PLAZA LIMA NORTE", "ESSALUD", "PLAZA SAN JOSÉ", "JESUS MARÍA",
    "REAL PLAZA SALAVERRY", "SAN FELIPE", "CAMACHO", "LA PLANICIE",
    "EL REMANSO", "LA MOLINA", "LA ROTONDA", "RINCONADA", "SEPARADORA INDUSTRIAL"
]


# Construct the new list with the desired strings
oficinas = ['Oficina BBVA ' + nombre for nombre in oficinas]

# Add the new column to the DataFrame
oficinas_bbva_lima['oficinas'] = oficinas
oficinas_bbva_lima

# Rearrange columns to place 'offices' at the top
column_order = ['oficinas'] + [col for col in oficinas_bbva_lima.columns if col != 'oficinas']
oficinas_bbva_lima = oficinas_bbva_lima[column_order]

oficinas_bbva_lima

,oficinas,Direccion,DEPARTAMENTO,PROVINCIA,DISTRITO
0,Oficina BBVA AEROPUERTO,CENTRO AEREO COMERCIAL LOCALES 110 A Y 111 A,LIMA,LIMA,CALLAO
1,Oficina BBVA BASE NAVAL,AV. CTRMTE. MORA S/N BASE NAVAL,LIMA,LIMA,CALLAO
2,Oficina BBVA C.C. CANTA CALLAO,"AV. ELMER FAUCETT Y ALEJANDRO BERTELLO, CC CAN...",LIMA,LIMA,CALLAO
3,Oficina BBVA CALLAO,AV. SAENZ PEN A 323,LIMA,LIMA,CALLAO
4,Oficina BBVA COLONIAL,CALLE OMEGA 149 PARQUE INDUSTRIAL DEL CALLAO,LIMA,LIMA,CALLAO
5,Oficina BBVA EL PUERTO,AV. CONTRALMIRANTE RAYGADA N°lll,LIMA,LIMA,CALLAO
6,Oficina BBVA LIMA CARGO CITY,"AV. ELMERT FAUCETT N°2121 LOCALES N° 2-101,2-1...",LIMA,LIMA,CALLAO
7,Oficina BBVA OVALO 200 MILLAS,AV. ELMER FAUCETT 6000,LIMA,LIMA,CALLAO
8,Oficina BBVA MINKA,CENTRO COMERCIAL MINKA PABELL6N 2,LIMA,LIMA,CALLAO
9,Oficina BBVA MALL AVENTURA PLAZA CALLAO,"AV. OSCAR R. BENAVIDES 3866, URB. EL AGUILA, L...",LIMA,LIMA,CALLAO
